<img src = "https://github.com/VeryFatBoy/notebooks/blob/main/common/images/img_github_singlestore-jupyter_featured_2.png?raw=true">

<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Using SingleStoreDB to map crimes and visualise hot routes</h1>
    </div>
</div>

In [4]:
!pip cache purge --quiet

In [5]:
!pip install geopandas --quiet

In [6]:
import pandas as pd
import geopandas as gpd
import warnings

from shapely import wkt
from shapely.geometry import LineString

warnings.filterwarnings("ignore")

In [7]:
bakerloo_stops_csv_url = "https://raw.githubusercontent.com/VeryFatBoy/singlestore-hot-routes-example/refs/heads/main/datasets/bakerloo_stops.csv"

df = pd.read_csv(bakerloo_stops_csv_url)

bakerloo_stops = gpd.GeoDataFrame(
    df["stn_name"],
    geometry = gpd.points_from_xy(df.stn_lon, df.stn_lat),
    crs = "EPSG:4326"
)

bakerloo_stops["geometry_wkt"] = bakerloo_stops["geometry"].apply(wkt.dumps)

bakerloo_stops_df = pd.DataFrame(
    bakerloo_stops.drop(columns = "geometry")
)

bakerloo_stops_df.rename(
    columns = {"geometry_wkt": "geometry"},
    inplace = True
)

In [8]:
bakerloo_stops_df.head(5)

,stn_name,geometry
0,Baker Street,POINT (-0.1571300000000000 51.5228830000000002)
1,Embankment,POINT (-0.1226660000000000 51.5070580000000007)
2,Edgware Road,POINT (-0.1701500000000000 51.5202990000000014)
3,Kilburn Park,POINT (-0.1942320000000000 51.5349789999999999)
4,Lambeth North,POINT (-0.1123150000000000 51.4988079999999968)


In [9]:
bakerloo_line_geojson_url = "https://raw.githubusercontent.com/VeryFatBoy/singlestore-hot-routes-example/refs/heads/main/datasets/bakerloo_line.geojson"

bakerloo_line = gpd.GeoDataFrame.from_file(bakerloo_line_geojson_url)
bakerloo_line.to_crs(epsg = 4326, inplace = True)

# https://stackoverflow.com/questions/62053253/how-to-split-a-linestring-to-segments
def segments(curve):
    return list(map(LineString, zip(curve.coords[:-1], curve.coords[1:])))

bakerloo_line_segments = segments(bakerloo_line.geometry[0])

bakerloo_sections = gpd.GeoDataFrame(
    geometry = bakerloo_line_segments,
    crs = "EPSG:4326"
)

bakerloo_sections["geometry_wkt"] = bakerloo_sections["geometry"].apply(wkt.dumps)

bakerloo_sections_df = pd.DataFrame(
    bakerloo_sections.drop(columns = "geometry")
)

bakerloo_sections_df.rename(
    columns = {"geometry_wkt": "geometry"},
    inplace = True
)

In [11]:
bakerloo_sections_df.head(5)

,geometry
0,LINESTRING (-0.3348960000000000 51.59221600000...
1,LINESTRING (-0.3169460000000000 51.58178600000...
2,LINESTRING (-0.3084480000000000 51.57022899999...
3,LINESTRING (-0.3039920000000000 51.56257999999...
4,LINESTRING (-0.2966420000000000 51.55232000000...


In [12]:
btp_street_csv_url = "https://raw.githubusercontent.com/VeryFatBoy/singlestore-hot-routes-example/refs/heads/main/datasets/2020-02-btp-street.csv"

df = pd.read_csv(btp_street_csv_url)

crimes = gpd.GeoDataFrame(
    df["Crime type"],
    geometry = gpd.points_from_xy(df.Longitude, df.Latitude),
    crs = "EPSG:4326"
)

crimes["geometry_wkt"] = crimes["geometry"].apply(wkt.dumps)

crimes_df = pd.DataFrame(
    crimes.drop(columns = "geometry")
)

crimes_df.rename(
    columns = {"geometry_wkt": "geometry", "Crime type" : "crime_type"},
    inplace = True
)

In [13]:
crimes_df.head(5)

,crime_type,geometry
0,Bicycle theft,POINT (-0.2716040000000000 50.8344000000000023)
1,Bicycle theft,POINT (-0.2716040000000000 50.8344000000000023)
2,Violence and sexual offences,POINT (-0.2371240000000000 50.8325000000000031)
3,Public order,POINT (-3.5594399999999999 54.6445000000000007)
4,Public order,POINT (-1.3308100000000000 53.0182000000000002)


In [14]:
bakerloo_line_buff = gpd.GeoDataFrame(
    geometry = bakerloo_line.buffer(0.005),
    crs = "EPSG:4326"
)

bakerloo_line_buff["geometry_wkt"] = bakerloo_line_buff["geometry"].apply(wkt.dumps)

bakerloo_line_buff_df = pd.DataFrame(
    bakerloo_line_buff[["geometry_wkt"]]
)

bakerloo_line_buff_df.rename(
    columns = {"geometry_wkt": "geometry"},
    inplace = True
)

In [15]:
bakerloo_line_buff_df.head(5)

,geometry
0,POLYGON ((-0.3144339861709700 51.5861091685744...


<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>Select the database from the drop-down menu at the top of this notebook. It updates the <b>connection_url</b> which is used by SQLAlchemy to make connections to the selected database.</p>
    </div>
</div>

In [16]:
from sqlalchemy import *

db_connection = create_engine(connection_url)

In [17]:
bakerloo_stops_df.to_sql(
    "bakerloo_stops",
    con = db_connection,
    if_exists = "append",
    index = False,
    chunksize = 1000
)

25

In [18]:
bakerloo_sections_df.to_sql(
    "bakerloo_sections",
    con = db_connection,
    if_exists = "append",
    index = False,
    chunksize = 1000
)

24

In [19]:
crimes_df.to_sql(
    "crimes",
    con = db_connection,
    if_exists = "append",
    index = False,
    chunksize = 1000
)

5361

In [20]:
bakerloo_line_buff_df.to_sql(
    "bakerloo_line_buff",
    con = db_connection,
    if_exists = "append",
    index = False,
    chunksize = 1000
)

1